# Web Scrapping using Python

In this project we would like to find all CSUF Faculty and Administration from 
CSUF 2020-2021 University Catalog at
   https://catalog.fullerton.edu/content.php?catoid=61&navoid=7316
using web scraping with Python. <font color=red>Note: There are 1274 Faculty and Staff in total</font>.

For each faculty or staff, we want fetch the following information:

- Name: Faculty or Staff name

- Year: the year of this faculty or staff joining CSUF
    
- Position: Position of this faculty or staff at CSUF
    
- Education: Degrees of this faculty or staff
    


**By using web scraping with Python we will extract the information 
of ``Name``, ``Year``, ``Position`` and ``Education`` for each faculty 
or staff. We will use the list ``Names`` to record all the names of all Faculty and Staff. The ith element 
of the list ``Names`` is
the name of the ith Faculty or Staff. The meaning of other lists ``Years``, 
``Positions`` and ``Educations`` are similar. If this information is missing, 
we will assign the value "N/A". For example, there is no education information for
the person "Brown, Alisha", then the value of ``Educations`` corrspodning to 
this person is "N/A".** 

In [3]:
import requests
import re
from bs4 import BeautifulSoup, NavigableString
import pandas as pd

url = 'https://catalog.fullerton.edu/content.php?catoid=61&navoid=7316'

page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
tds=soup.find('td', class_='block_content_outer')
all_ps=tds.find_all('p')

Names=[]
Years=[]
Positions=[]
Educations=[]


for info in all_ps :
   
    #name
    name_year = info.find('strong')
    if name_year is not None:
        name = re.findall('[a-zA-Z]+', name_year.text)
        if len(name[0]) > 1:
            Names.append(' '.join(name))
        
        #year
            year = re.findall('[0-9]+', name_year.text)
            if len(year) > 0:
                Years.append(year[0])
            else:
                Years.append('N/A')
                
        # position
            position=info.find('em')
            if position is not None:
                Positions.append(position.text)
            else:
                Positions.append('N/A')
                
            Positions = [el.replace('\xa0',' ') for el in Positions]

           
        #education    
            e = info.get_text()
            if e is None:
                continue
            E = e.split("\n")
            del E[0]
            if not E:
                E.append('N/A')

            E1 = E[0]
            Educations.append(E1)
    
            Educations = [el.replace('\xa0',' ') for el in Educations]
            
            
            



print(Names[:5])
print('\n')
print(Years[:5])
print('\n')
print(Positions[:5])
print('\n')
print(Educations[:5])


['Virjee Framroze M', 'Abdoli Mansour', 'Abdou Noha', 'Abedi Vahideh S', 'Abnet Dustin A']


['2018', '2017', '2018', '2013', '2014']


['President', 'Lecturer in Mathematics', 'Associate Director, Institutional Research', 'Associate Professor of Information Systems and Decision Sciences', 'Assistant Professor of American Studies ']


['B.A. University of California, Santa Barbara; J.D. University of California, Hastings College of the Law', 'M.S. California State University, Fullerton', 'B.A. American University in Cairo; M.P.A. California State Polytechnic University, Pomona; Ed.D. California State University, Fullerton', 'Ph.D. University of Toronto, School of Graduate Studies', 'B.A., M.A. Miami University; Ph.D. Indiana University']


**Now we will merge the lists ``Names``, ``Years``,  ``Positions`` and ``Educations`` into a pandas 
DataFrame ``FacultyStaff2020to2021``. 
Then we will Put the list ``Names`` in the first column
of the DataFrame ``FacultyStaff2020to2021``, 
the list ``Years`` in the second column,
the list ``Positions`` in the third column,
and the list ``Educations`` in the fourth column.
We will use Name, Year, Position, and Education for the 
column names of the DataFrame ``FacultyStaff2020to2021``. 
Then we will display the first five row of this DataFrame.**

<img src="https://drive.google.com/uc?export=view&id=1KJedbcW2zF0kfzbtSmNYfZQyZTMnsDBO" width="750"></img>

In [4]:
FacultyStaff2020to2021 = pd.DataFrame({'Name': Names,
                                       'Year': Years,
                                       'Position': Positions,
                                       'Education': Educations
    
})
print(FacultyStaff2020to2021.info())

FacultyStaff2020to2021.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1239 entries, 0 to 1238
Data columns (total 4 columns):
Name         1239 non-null object
Year         1239 non-null object
Position     1239 non-null object
Education    1239 non-null object
dtypes: object(4)
memory usage: 38.8+ KB
None


,Name,Year,Position,Education
0,Virjee Framroze M,2018,President,"B.A. University of California, Santa Barbara; ..."
1,Abdoli Mansour,2017,Lecturer in Mathematics,"M.S. California State University, Fullerton"
2,Abdou Noha,2018,"Associate Director, Institutional Research",B.A. American University in Cairo; M.P.A. Cali...
3,Abedi Vahideh S,2013,Associate Professor of Information Systems and...,"Ph.D. University of Toronto, School of Graduat..."
4,Abnet Dustin A,2014,Assistant Professor of American Studies,"B.A., M.A. Miami University; Ph.D. Indiana Uni..."


**Now we will add one column ``Faculty or Staff`` after the 
column ``Education`` in the DataFrame ``FacultyStaff2020to2021``. 
The value in this column can be "Faculty" or "Staff". 
We will use the following rules to classify one employer into either
"Faculty" or "Staff": One employer is Facuty if his/her position 
contains the word "Lecturer" or "Professor". Otherwise, this employer
is "Staff". Then we will display the first five row of this DataFrame.**

<img src="https://drive.google.com/uc?export=view&id=1g02CaKH4SLFJLNbsSq0IYXSNXdYp84wI" width="750"></img>

In [5]:
Staff_Faculty=[]
for values in  FacultyStaff2020to2021["Position"]:
    if "Lecturer" in values:
        Staff_Faculty.append('Faculty')
     
    elif "Professor" in values:
        Staff_Faculty.append('Faculty')
    
    else:
        Staff_Faculty.append('Staff')
        

        
        
        
# Define The Column.
        
FacultyStaff2020to2021['Faculty/Staff'] = Staff_Faculty

FacultyStaff2020to2021





,Name,Year,Position,Education,Faculty/Staff
0,Virjee Framroze M,2018,President,"B.A. University of California, Santa Barbara; ...",Staff
1,Abdoli Mansour,2017,Lecturer in Mathematics,"M.S. California State University, Fullerton",Faculty
2,Abdou Noha,2018,"Associate Director, Institutional Research",B.A. American University in Cairo; M.P.A. Cali...,Staff
3,Abedi Vahideh S,2013,Associate Professor of Information Systems and...,"Ph.D. University of Toronto, School of Graduat...",Faculty
4,Abnet Dustin A,2014,Assistant Professor of American Studies,"B.A., M.A. Miami University; Ph.D. Indiana Uni...",Faculty
...,...,...,...,...,...
1234,Zettel Watson Laura,2005,"Professor of Psychology; Coordinator, Aging S...",B.A. University of Michigan; Ph.D. University ...,Faculty
1235,Zhang Fang,2012,Associate Professor of Economics,"B.A. Zhejiang University; M.A., Ph.D. The Ohio...",Faculty
1236,Zhang Yi Jenny,2004,"Associate Dean, Academic Programs and Student ...",B.S. University of Science and Technology of C...,Faculty
1237,Zhou Jie,2016,Associate Professor of Accounting,Ph.D. Northwestern University,Faculty


**Now we will compute the number of new faculty or staff 
members for each year in the period of 2010-2020. We will create a data 
frame named ``NewFacultyStaff`` including three coulumns Year, New_Facuty_Num and 
New_Staff_Num where the column Year takes value from 2010 to 2020.
Each row of this data frame records the number of new faculty members
and the number of new staff members for one particular year.**

The first five row of our DataFrame should look like the following:
 
<img src="https://drive.google.com/uc?export=view&id=1-jF4rkadLLttaseMyoO-VzU-zPe6mJZN" width="400"></img>

In [6]:
from collections import Counter

New_Faculty_Num = []
New_Staff_Num = []

Faculty_Year=FacultyStaff2020to2021.loc[(FacultyStaff2020to2021["Faculty/Staff"] == "Faculty") 
                                        & (FacultyStaff2020to2021["Year"] > '2009')]
Year=Faculty_Year["Year"]
for f_year in Year:
    if f_year != 'N/A':
        New_Faculty_Num.append(f_year)
faculty_count = Counter(New_Faculty_Num)

Staff_Year=FacultyStaff2020to2021.loc[(FacultyStaff2020to2021["Faculty/Staff"] == "Staff") 
                                        & (FacultyStaff2020to2021["Year"] > '2009')]
Years=Staff_Year["Year"]
for s_year in Years:
    if s_year != 'N/A':
        New_Staff_Num.append(s_year)
staff_count = Counter(New_Staff_Num)

faculty_df = pd.DataFrame.from_dict(faculty_count, orient='index').reset_index()
staff_df = pd.DataFrame.from_dict(staff_count, orient='index').reset_index()


NewFacultyStaff = pd.merge(faculty_df, staff_df, on="index")
NewFacultyStaff.sort_values('index')

NewFacultyStaff.columns = ['Year','New_Faculty_Num','New_Staff_Num' ]
print(NewFacultyStaff.columns) 
NewFacultyStaff = NewFacultyStaff.sort_values(by ='Year' )
NewFacultyStaff.set_index('Year', inplace=True)
 
NewFacultyStaff

Index(['Year', 'New_Faculty_Num', 'New_Staff_Num'], dtype='object')


,New_Faculty_Num,New_Staff_Num
Year,,
2010,23,3
2011,39,7
2012,29,13
2013,39,7
2014,44,11
2015,69,41
2016,62,27
2017,45,42
2018,27,36


**Finally we will export our DataFrame ``FacultyStaff2020to2021`` to a CSV file**

In [8]:
FacultyStaff2020to2021.to_csv(r'Web_Scrapping.csv', index = False)